In [1]:
import configparser
import datetime as dt
import sys
import collections
from pathlib import Path
from typing import Union, Optional, List, Tuple

import codebook.EDA as EDA
import codebook.clean as clean
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy

In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
plt.style.use('raph-base')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 30)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

np.random.seed(666)

In [3]:
Path.cwd().parent / 'src'

WindowsPath('C:/Users/r2d4/OneDrive/code/projects/20-02_disco/src')

In [4]:
sys.path.append(str(Path.cwd().parent))

from src.db_declaration import (
    Base, Artist, CreditTrx, Genre, Label, Record, RecordFormat, CreditTrx, RecordLabelLink, ArtistLabelLink
)
from src import db_functions
from src import utils

In [5]:
print(sys.executable)
print(sys.version)
print(f'sqlalchemy {sqlalchemy.__version__}')

C:\Users\r2d4\miniconda3\envs\py3\python.exe
3.8.3 (default, May 19 2020, 06:50:17) [MSC v.1916 64 bit (AMD64)]
sqlalchemy 1.3.17


## Connect To DB

In [6]:
engine = utils.create_engine(r"C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase.db")
session = utils.create_session(engine)

C:\Users\r2d4\OneDrive\code\projects\20-02_disco\db_aka_discobase\DiscoBase.db


## Query DB

In [18]:
pd.read_sql("records", engine).sample(3)

,record_id,artist_id,title,genre_id,label,year,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,digitized,rating,active,created_at,updated_at
154,155,88,Cult Of The Goat,3,None,2018,None,None,400,None,out of 500,2018-02-01,20,0,nan,1,2021-01-09 11:40:25,NaT
104,105,53,Deicide,1,None,1990,None,None,None,None,None,2017-04-01,50,1,nan,1,2021-01-09 11:40:23,NaT
230,231,141,Obscene Majesty,1,None,2019,None,obxblood-silver merge,300,None,"Limited to 300 copies, hype sticker is incorrect (says 200).",2019-08-01,25,1,6.00,1,2021-01-09 11:40:28,NaT


In [7]:
pd.read_sql("records", engine).tail()

,record_id,artist_id,title,genre_id,label,year,format_id,vinyl_color,lim_edition,number,remarks,purchase_date,price,digitized,rating,active,created_at,updated_at
292,293,188,We Move As One,5,None,2020,None,clear,None,None,None,2020-12-01,20,1,nan,1,2021-01-09 11:40:30,NaT
293,294,189,Crypt Of Ice,1,None,2021,None,lilac,200,None,None,2020-12-12,25,0,nan,1,2021-01-09 11:42:41,NaT
294,295,190,Ratlord,1,None,2017,None,"random colored, w/ b-side silkscreenprint in white",lim,None,None,2021-01-07,15,0,nan,1,2021-01-09 11:46:04,NaT
295,296,191,I Am Your God,5,None,2020,None,white / black,200,None,None,2021-01-09,20,1,nan,1,2021-01-09 12:47:46,NaT
296,297,192,Anti Life Anti You,5,None,2020,None,orange with black swirl,200,None,None,2021-01-09,20,1,nan,1,2021-01-09 12:50:27,NaT


In [8]:
pd.read_sql("records", engine)["active"].sum()

280

In [9]:
pd.read_sql("artists", engine).tail

<bound method NDFrame.tail of      artist_id       artist_name artist_country          created_at updated_at
0            1         Dismember           None 2021-01-09 11:40:20        NaT
1            2         Merciless           None 2021-01-09 11:40:20        NaT
2            3          Overkill           None 2021-01-09 11:40:20        NaT
3            4          Entombed           None 2021-01-09 11:40:20        NaT
4            5         Metallica           None 2021-01-09 11:40:20        NaT
..         ...               ...            ...                 ...        ...
187        188        Ironed Out           None 2021-01-09 11:40:30        NaT
188        189       Frozen Soul            USA 2021-01-09 11:42:41        NaT
189        190           Ratlord        Germany 2021-01-09 11:46:04        NaT
190        191              Xile    New Zealand 2021-01-09 12:47:46        NaT
191        192  Dead End Tragedy        Germany 2021-01-09 12:50:27        NaT

[192 rows x 5 columns

In [10]:
pd.read_sql("credit_trx", engine).tail()

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
295,296,2021-01-07,Purchase,-1.00,-2.00,295.00,2021-01-09 11:46:04,NaT
296,297,2021-01-09,Addition,1.00,-1.00,nan,2021-01-09 11:51:46,NaT
297,298,2021-01-09,Purchase,-1.00,-2.00,296.00,2021-01-09 12:47:46,NaT
298,299,2021-01-09,Purchase,-1.00,-3.00,297.00,2021-01-09 12:50:27,NaT
299,300,2021-01-09,Removal,0.00,-3.00,83.00,2021-01-09 21:13:27,NaT


In [11]:
pd.read_sql("record_label_link", engine).tail()
pd.read_sql("artist_label_link", engine).tail()
pd.read_sql("artist_genre_link", engine).tail()
pd.read_sql("genre_label_link", engine).tail()

,record_id,label_id
292,293,41
293,294,6
294,295,42
295,296,23
296,297,23


,artist_id,label_id
198,188,41
199,189,6
200,190,42
201,191,23
202,192,23


,artist_id,genre_id
189,188,5
190,189,1
191,190,1
192,191,5
193,192,5


,genre_id,label_id
46,5,38
47,5,39
48,5,40
49,5,41
50,1,42


In [12]:
pd.read_sql("labels", engine).sort_values("label_name")

,label_id,label_name,created_at,updated_at
14,15,20 Buck Spin,2021-01-09 11:40:27,NaT
23,24,6131 Records,2021-01-09 11:40:28,NaT
17,18,BBB Records,2021-01-09 11:40:27,NaT
22,23,BDHW Records,2021-01-09 11:40:28,NaT
19,20,Back on Black,2021-01-09 11:40:28,NaT
29,30,Blood Harvest,2021-01-09 11:40:29,NaT
15,16,Bridge 9 Records,2021-01-09 11:40:27,NaT
13,14,Carbonized Records,2021-01-09 11:40:27,NaT
5,6,Century Media,2021-01-09 11:40:23,NaT
37,38,Closed Casket Activities,2021-01-09 11:40:30,NaT


In [23]:
result = session.query(Record).join(Artist).filter(Artist.artist_name == "Furia").all()
for record in result:
    print(record.labels, record.format_id)

[<Label(label_id=1, label_name=NA)>] None
[<Label(label_id=1, label_name=NA)>] None
[<Label(label_id=1, label_name=NA)>] None


## Delete A Record (Temporary Section)

In [14]:
# removal = {
#     "trx_type": "Removal",
#     "credit_value": 0,
#     "artist": "Furia",
#     "title": "Marzannie, Królowej Polski",
#     "year": 2012,
#     "date": dt.datetime.today().date()
# }

In [15]:
# db_functions.set_record_to_inactive(session, removal)

Status of record 'Marzannie, Królowej Polski' by Furia is already 0, please check.


In [16]:
# pd.read_sql("credit_trx", engine).tail()

,credit_trx_id,credit_trx_date,credit_trx_type,credit_value,credit_saldo,record_id,created_at,updated_at
295,296,2021-01-07,Purchase,-1.00,-2.00,295.00,2021-01-09 11:46:04,NaT
296,297,2021-01-09,Addition,1.00,-1.00,nan,2021-01-09 11:51:46,NaT
297,298,2021-01-09,Purchase,-1.00,-2.00,296.00,2021-01-09 12:47:46,NaT
298,299,2021-01-09,Purchase,-1.00,-3.00,297.00,2021-01-09 12:50:27,NaT
299,300,2021-01-09,Removal,0.00,-3.00,83.00,2021-01-09 21:13:27,NaT


In [17]:
# to_delete = session.query(CreditTrx).filter(CreditTrx.credit_trx_id == 301).one()
# session.delete(to_delete)
# session.commit()

NoResultFound: No row was found for one()